In [1]:
import random
import json
import re
import os
import bz2
import spacy
nlp = spacy.load('en', disable=['ner', 'textcat', 'depend'])

# Functions to read and prepare data

In [2]:
# remove some frequent garbage from sentences 
def preprocessSentence(text):
    text = re.sub('-LRB-(.*?)-RRB-', '', text)
    text = re.sub('-RRB-', '', text)
    text = re.sub('^\"', '', text)
    text = re.sub('\"$', '', text)
    text = re.sub('\\n', '', text)
    text = re.sub('\\r', '', text)
    text = re.sub('\\t', '', text)
    text = re.sub('\`', '', text)
    text = re.sub('\'\'', '', text)
    return text

In [3]:
# read Newsela dataset
def readNewselaData(compl_to_sim):
    with bz2.BZ2File('../dataset/clear_newsela.txt.bz2', "r") as content:         
        for line in content:
            texts = line.split('\t')

            if len(texts) != 3:
#                 print 'wrong line: ', len(texts), line
                continue
            if (len(texts[0]) < 10) or (len(texts[1]) < 10):
#                 print 'empty line: ', line
                continue
                
            compl_to_sim[preprocessSentence(texts[0])] = preprocessSentence(texts[1])
    return compl_to_sim

In [4]:
def readWikiData(compl_to_sim):
    complex_sentences = {}
    with bz2.BZ2File('../dataset/wiki.bz2', "r") as content:  
        i = 1
        for line in content:
            complex_sentences[i] = preprocessSentence(line)
            i += 1
    i = 1
    with bz2.BZ2File('../dataset/simple_wiki.bz2', "r") as simple:
        for line in simple:
            prep_line = preprocessSentence(line)

            try:
                if (complex_sentences[i] != prep_line) or (len(complex_sentences[i]) < 100):
                    compl_to_sim[complex_sentences[i]] = prep_line
            except:
                continue
            i += 1
    return compl_to_sim

In [5]:
def readPWKPData(compl_to_sim):
    complex_sentences = {}
    with bz2.BZ2File('../dataset/PWKP_108016.bz2', "r") as content:  
        simple = False
        simple_sent = ''
        compls_sent = ''
#         i = 0
        for line in content:
#             i+=1
#             if i > 20:
#                 break
            if len(line) < 5:
                simple = False
                if (len(simple_sent) > 10) and (len(compls_sent) > 10):
#                     print 'add sentences'
                    compl_to_sim[preprocessSentence(compls_sent)] = preprocessSentence(simple_sent)
                simple_sent = ''
                compls_sent = ''
            elif simple == False:
                compls_sent += line + ' '
                simple = True
            else:
                simple_sent += line + ' '
    return compl_to_sim

In [16]:
def clearSentence(text, vocab):
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    text = text.decode('UTF-8')
    text = text.split()
    text = [word for word in text if not word.isdigit()]
    text = [word.lower() for word in text]
    text = [re_print.sub('', w) for w in text]
    
    for word in text:
        if word not in vocab:
            vocab.append(word)
    tokens_count = len(text)
    return ' '.join(text), tokens_count, vocab
    

In [17]:
# split data to train and test set
def splitTrainTest(compl_to_sim):
    train = {}
    test = {}
    com_vocab = []
    sim_vocab = []
    max_com_length = 0
    max_sim_length = 0

    for compl, sim in compl_to_sim.iteritems():
        try:
            sim_u = sim.encode('utf-8')
            compl_u = compl.encode('utf-8')
        except:
            continue
#         com, s = replaceProperNouns(compl, sim)
#         com, s = compl, sim
        com, lc, com_vocab = clearSentence(compl, com_vocab)
        if (lc > 50) or (lc < 5):
            continue
        s, ls, sim_vocab = clearSentence(sim, sim_vocab)
        if (ls < 5) or (ls/float(lc) > 2.0) or (ls/float(lc) < 0.5): 
            continue
        
        if (max_com_length < lc):
            max_com_length = lc
        if (max_sim_length < ls):
            max_sim_length = ls
            
        r = random.random()
        if r < 0.8:
            train[com] = s
        else:
            test[com] = s
    return train, test, com_vocab, sim_vocab, max_com_length, max_sim_length

# Read and preprocess data

In [8]:
compl_to_sim = {}

In [9]:
compl_to_sim = readNewselaData(compl_to_sim)

In [10]:
compl_to_sim = readWikiData(compl_to_sim)

In [11]:
compl_to_sim = readPWKPData(compl_to_sim)

In [12]:
len(compl_to_sim)

234022

In [15]:
random.choice(list(compl_to_sim.items()))

('This combination of propellants is sometimes still used in torpedoes. ',
 'This combination of propellants is still used in torpedoes. ')

In [14]:
import string

In [18]:
# train, test = splitTrainTest(compl_to_sim)
# print len(train), len(test)

train, test, encoder_tokens, decoder_tokens, max_encoder_seq_length, max_decoder_seq_length \
= splitTrainTest(compl_to_sim)
num_encoder_tokens = len(encoder_tokens)
num_decoder_tokens = len(decoder_tokens)

KeyboardInterrupt: 

In [ ]:
print('Number of samples:', len(train), len(test))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

In [ ]:
with open("../dataset/train.txt", "w") as f:
    for com, sim in train.iteritems():
        f.write(com + '\t' + sim)

with open("../dataset/test.txt", "w") as f:
    for com, sim in test.iteritems():
        f.write(com + '\t' + sim)
        
with open("../dataset/com_vocab.txt", "w") as f:
    for tok in encoder_tokens:
        f.write(tok)
        
with open("../dataset/sim_vocab.txt", "w") as f:
    for tok in decoder_tokens:
        f.write(tok)

In [17]:
random.choice(list(train.items()))    

('In 1913 he published his first collection of poems , The Stone  ; it was reissued in 1916 under the same title , but with additional poems included .',
 'At the same time his second book of poems , Tristia , was published in Berlin . For several years after that , he almost completely abandoned poetry , concentrating on essays , literary criticism , memoirs  and small-format prose  .')

In [ ]:
'''Sequence to sequence example in Keras (character-level).
This script demonstrates how to implement a basic character-level
sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.
# Summary of the algorithm
- We start with input sequences from a domain (e.g. English sentences)
    and correspding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    Is uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.
'''

In [ ]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [ ]:
batch_size = 64  # Batch size for training.
epochs = 10  # Number of epochs to train for. ___100___
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

In [ ]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(encoder_tokens)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(decoder_tokens)])

In [ ]:
encoder_input_data = np.zeros(
    (len(train), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(train), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(train), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [ ]:
for i, (input_text, target_text) in enumerate(train):
    for t, char in input_text.split():
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

